In [1]:
import os
import random

from dotenv import load_dotenv
from pyspark.sql import SparkSession

load_dotenv()

os.environ["PYSPARK_SUBMIT_ARGS"] = (
            "--packages org.apache.hadoop:hadoop-aws:3.3.4 pyspark-shell"
        )

# Inicializar a sessão Spark
spark = SparkSession.builder \
    .appName("ParquetReaderExample") \
    .config("spark.hadoop.fs.s3a.endpoint", os.environ["MINIO_ENDPOINT"]) \
    .config("spark.hadoop.fs.s3a.access.key", os.environ["MINIO_ACCESS_KEY"]) \
    .config("spark.hadoop.fs.s3a.secret.key", os.environ["MINIO_SECRET_KEY"]) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .getOrCreate()  # noqa: E501


your 131072x1 screen size is bogus. expect trouble
25/01/15 11:11:08 WARN Utils: Your hostname, CPC-12806 resolves to a loopback address: 127.0.1.1; using 172.26.242.248 instead (on interface eth0)
25/01/15 11:11:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/arthur/minio-python-client/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/arthur/.ivy2/cache
The jars for the packages stored in: /home/arthur/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0291f313-9446-425d-84a7-852a6626dd50;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 296ms :: artifacts dl 15ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	--------------------------------

In [5]:
# Criar um DataFrame com dados aleatórios simulando um cenário real
data = [(i, f"Categoria_{random.randint(1, 5)}", random.uniform(10.0, 500.0)) for i in range(200)]
columns = ["id", "categoria", "valor"]

df = spark.createDataFrame(data, schema=columns)

# Exibir os primeiros registros
df.show(5)

+---+-----------+------------------+
| id|  categoria|             valor|
+---+-----------+------------------+
|  0|Categoria_4| 141.5955612049314|
|  1|Categoria_2|232.88568243203932|
|  2|Categoria_1| 80.34393746959496|
|  3|Categoria_5|359.03091052980693|
|  4|Categoria_5|150.48462611896738|
+---+-----------+------------------+
only showing top 5 rows



In [6]:
# Salvar o DataFrame no MinIO em formato Parquet particionado pela coluna 'categoria'
output_path = "s3a://sandbox/pyspark_example/"
df.write.mode("overwrite").parquet(output_path)

25/01/15 11:14:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/01/15 11:14:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/01/15 11:14:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/01/15 11:14:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/01/15 11:14:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/01/15 11:14:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/01/15 11:14:10 WARN MemoryManager: Total allocation exceeds 95.

In [8]:
# Caminho para os arquivos Parquet
parquet_path = "s3a://sandbox/pyspark_example/*.parquet"

# Ler todos os arquivos Parquet
df = spark.read.parquet(parquet_path)

# Exibir os primeiros registros
df.show(5)

+---+-----------+------------------+
| id|  categoria|             valor|
+---+-----------+------------------+
|176|Categoria_2|497.30971569959235|
|177|Categoria_3| 325.7741928192519|
|178|Categoria_2|116.64221565713207|
|179|Categoria_2|490.02376605378424|
|180|Categoria_3|456.00108822969884|
+---+-----------+------------------+
only showing top 5 rows



In [9]:
# Transformação simples: filtrar registros onde a coluna 'valor' > 100
df_filtered = df.filter(df["valor"] > 100)

# Exibir o DataFrame filtrado
df_filtered.show(5)

+---+-----------+------------------+
| id|  categoria|             valor|
+---+-----------+------------------+
|176|Categoria_2|497.30971569959235|
|177|Categoria_3| 325.7741928192519|
|178|Categoria_2|116.64221565713207|
|179|Categoria_2|490.02376605378424|
|180|Categoria_3|456.00108822969884|
+---+-----------+------------------+
only showing top 5 rows



In [10]:
# Registrar o DataFrame como uma tabela temporária
df.createOrReplaceTempView("dados")

# Executar a consulta SQL para filtrar registros com valor > 100
df_filtered_sql = spark.sql("""
    SELECT *
    FROM dados
    WHERE valor > 100
""")

# Exibir o resultado da filtragem
df_filtered_sql.show(5)


+---+-----------+------------------+
| id|  categoria|             valor|
+---+-----------+------------------+
|176|Categoria_2|497.30971569959235|
|177|Categoria_3| 325.7741928192519|
|178|Categoria_2|116.64221565713207|
|179|Categoria_2|490.02376605378424|
|180|Categoria_3|456.00108822969884|
+---+-----------+------------------+
only showing top 5 rows

